# 03c03 Smoothness

In [ ]:
# default_exp embed
# hide
from nbdev.showdoc import *
import numpy as np
import matplotlib.pyplot as plt
import torch
import FRED
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
%load_ext autoreload
%autoreload 2

We want our vector field to be as simple as possible. One means of achieving this is to regularize it by smoothness, to penalize it for changing the direction of the vectors too suddenly. Here, I'll code up a grid-based laplacian normalizer. It will take samples from a grid graph in the support of the points and flow, calculate the  laplacian of the grid, and compute the smoothness via
$$ \frac{v^T L v}{v^T v} $$

In [ ]:
#export
def anisotropic_kernel(D, sigma=0.7, alpha = 1):
  """Computes anisotropic kernel of given distances matrix.

  Parameters
  ----------
  D : ndarray or sparse
  sigma : float, optional
      Kernel bandwidth, by default 0.7
  alpha : int, optional
      Degree of density normalization, from 0 to 1; by default 1
  This is a good function.
  """
  W = torch.exp(-D**2/(2*sigma**2))
  # Additional normalization step for density
  D = torch.diag(1/(torch.sum(W,axis=1)**alpha)) 
  W = D @ W @ D
  return W

In [ ]:
# export
def smoothness_of_vector_field(embedded_points, vector_field_function, device, use_grid = True, grid_width = 20):
    if use_grid:
        # find support of points
        minx = (min(embedded_points[:,0])-1).detach()
        maxx = (max(embedded_points[:,0])+1).detach()
        miny = (min(embedded_points[:,1])-1).detach()
        maxy = (max(embedded_points[:,1])+1).detach()
        # form grid around points
        x, y = torch.meshgrid(torch.linspace(minx,maxx,steps=grid_width),torch.linspace(miny,maxy,steps=grid_width))
        xy_t = torch.concat([x[:,:,None],y[:,:,None]],dim=2).float()
        xy_t = xy_t.reshape(grid_width**2,2).to(device)
        points_to_test = xy_t
    else:
        points_to_test = embedded_points
    # Compute distances between points
    # TODO: Can compute A analytically for grid graph, don't need to run kernel
    Dists = torch.cdist(points_to_test,points_to_test)
    A = anisotropic_kernel(Dists)
    # Get degree matrix and build graph laplacian
    D = A.sum(axis=1)
    L = torch.diag(D) - A
    # compute vector field at each grid point
    vecs = vector_field_function(points_to_test)
    x_vecs = vecs[:,0]
    y_vecs = vecs[:,1]
    # compute smoothness of each x and y and add them # TODO: There are other ways this could be done
    x_smoothness = (x_vecs.T @ L @ x_vecs) / torch.max(torch.linalg.norm(x_vecs)**2, torch.tensor(1e-5))
    y_smoothness = (y_vecs.T @ L @ y_vecs) / torch.max(torch.linalg.norm(y_vecs)**2, torch.tensor(1e-5))
    total_smoothness = x_smoothness + y_smoothness
    return total_smoothness